In [ ]:
import cv2
import numpy as np

# 参照画像(img_ref)と比較画像(img_cmp)の読み込み
img_ref = cv2.imread('xxxxx.png', cv2.IMREAD_COLOR)
img_cmp = cv2.imread('xxxxx.png', cv2.IMREAD_COLOR)
img_tmp = img_cmp.copy()

# グレースケール変換
gray_img_ref = cv2.cvtColor(img_ref, cv2.COLOR_BGR2GRAY)
gray_img_cmp = cv2.cvtColor(img_cmp, cv2.COLOR_BGR2GRAY)

# 参照画像の平滑化 (変化をつけるために意図的に実行する)
gray_img_ref = cv2.blur(gray_img_ref, (3,3))

# 単純に画像の引き算
img_dif = cv2.absdiff(gray_img_ref, gray_img_cmp)

# 差分画像を2値化 (閾値は50)
(ret,img_bin) = cv2.threshold(img_dif, 50, 255, 0)

# 2値画像に存在する輪郭の座標を取得
(contours, hierarchy) = cv2.findContours(img_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# contoursから1個ずつ輪郭を取り出し、輪郭の位置(x,y)とサイズ(cx,cy)を取得
# サイズが 5x5 以上の輪郭を枠で囲う
for contour in contours:
    (x,y,cx,cy) = cv2.boundingRect(contour)
    if cx > 5 and cy > 5:
        img1 = gray_img_ref[y-2:y+cy+2, x-2:x+cx+2]
        img2 = gray_img_cmp[y-2:y+cy+2, x-2:x+cx+2]
        (d,etc) = cv2.phaseCorrelate(np.float32(img2),np.float32(img1))
        (dx,dy) = d
        (dx,dy) = (np.around(np.abs(dx)), np.around(np.abs(dy)))
        print('dx=', dx, ' dy=', dy)
        if dx > 1 or dy > 1:
            cv2.rectangle(img_tmp, (x-2, y-2), (x+cx+2, y+cy+2), (0, 255, 0), 1)

# 画像保存
# cv2.imwrite('xxxxx-diff.png', img_dif)
# cv2.imwrite('xxxxx-bin.png', img_bin)
cv2.imwrite('xxxxx-result.png', img_tmp)